In [1]:
import pandas as pd
import pymysql.cursors
import numpy as np

In [4]:
sql="select customers.customerName, payments.checkNumber from customers left join payments on customers.customerNumber=payments.customerNumber where payments.paymentDate >= '2004-10-06';"


In [5]:
# MySQL DB에서 데이터 받아와서 DataFrame에 저장

conn = pymysql.connect(host='localhost', user='root', 
                       password='7756', db='tip', charset='utf8',
                       autocommit=True, cursorclass=pymysql.cursors.DictCursor)

In [6]:
try:

   with conn.cursor() as curs:
      sql="SELECT * FROM tips;"
      curs.execute(sql)
      rs = curs.fetchall()

      # DB에서 받아온 값을 DataFrame에 넣음

      df = pd.DataFrame(rs)
      print(df)

finally:

   conn.close()

     total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
248       10.34  1.66    Male     No   Sun  Dinner     3
249       21.01  3.50    Male     No   Sun  Dinner     3
250       21.01  3.50    Male     No        Dinner     3
251       16.99  1.01  Female     No  None  Dinner     2
252       10.34  1.66    None     No   Sun  Dinner     3

[253 rows x 7 columns]


In [7]:
df['tip_rate']=df.tip/df.total_bill

In [8]:
df.head()

,total_bill,tip,sex,smoker,day,time,size,tip_rate
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [9]:
df.isnull().sum()

total_bill    0
tip           0
sex           1
smoker        0
day           1
time          0
size          0
tip_rate      0
dtype: int64

In [26]:
df.sex.replace({'Female':0,'Male':1},inplace=True)

In [27]:
df.smoker.replace({'No':0,'Yes':1},inplace=True)
df.day.replace({'Thur':0,'Fri':1,'Sat':2,'Sun':3},inplace=True)
df.time.replace({'Lunch':0,'Dinner':1},inplace=True)

In [28]:
df.replace('',np.nan,inplace=True) 

In [29]:
df.head()

,total_bill,tip,sex,smoker,day,time,size,tip_rate
0,16.99,1.01,0.0,0,3.0,1,2,0.059447
1,10.34,1.66,1.0,0,3.0,1,3,0.160542
2,21.01,3.50,1.0,0,3.0,1,3,0.166587
3,23.68,3.31,1.0,0,3.0,1,2,0.139780
4,24.59,3.61,0.0,0,3.0,1,4,0.146808


In [14]:
df.isnull().sum()

total_bill    0
tip           0
sex           1
smoker        0
day           2
time          0
size          0
tip_rate      0
dtype: int64

In [15]:
type(df)

pandas.core.frame.DataFrame

In [16]:
# from numpy import isnan
from sklearn.impute import SimpleImputer

In [17]:
data = df.values

In [18]:
df.describe()

,total_bill,tip,sex,smoker,day,time,size,tip_rate
count,253.000000,253.000000,252.000000,253.000000,251.000000,253.000000,253.000000,253.000000
mean,19.655296,2.964783,0.642857,0.367589,1.760956,0.731225,2.573123,0.159666
std,8.807863,1.384318,0.480111,0.483104,1.158719,0.444201,0.938373,0.060975
min,3.070000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.035638
25%,13.280000,2.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.128244
50%,17.590000,2.830000,1.000000,0.000000,2.000000,1.000000,2.000000,0.155581
75%,24.010000,3.500000,1.000000,1.000000,3.000000,1.000000,3.000000,0.188395
max,50.810000,10.000000,1.000000,1.000000,3.000000,1.000000,6.000000,0.710345


In [19]:
# flatten() 여러 칼럼을 하나로 합친다. 
sum(np.isnan(data).flatten())

3

In [20]:
imputer = SimpleImputer(strategy='median')

In [21]:
# fit on the dataset
imputer.fit(data)

SimpleImputer(strategy='median')

In [22]:
# transform the dataset
data_trans = imputer.transform(data)

In [23]:
df_trans=pd.DataFrame(data_trans)

In [24]:
df_trans.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
dtype: int64

In [30]:
### RFE

from sklearn.feature_selection import RFE

#from sklearn.tree import DecisionTreeClassifier

from sklearn.svm import SVR

estimator = SVR(kernel="linear")

# define RFE
rfe = RFE(estimator, n_features_to_select=4)

# fit RFE
selector=rfe.fit(data_trans, y)

!cd

selector.support_

# summarize all features
for i in range(data_trans.shape[1]):
  print('Column: %d,  Rank: %d' % (i,  rfe.ranking_[i]))

df1=pd.DataFrame(data_trans, columns=['total_bill', 'sex', 'smoker', 'day', 'time', 'size'])

df1.columns

df1.drop(['smoker', 'day'], axis=1)

NameError: name 'y' is not defined

In [31]:
### PCA

from sklearn.decomposition import PCA

data_trans.shape

# define the transform
trans = PCA(n_components=4)

# transform the data
X_dim = trans.fit_transform(data_trans)

# summarize data after the transform
print(X_dim[:3, :])



[[-2.86776715 -1.17667997  1.74910005 -0.02218638]
 [-9.32484501 -1.58826821  0.12742297  0.97026726]
 [ 1.45639783 -1.30231053 -0.41007166  0.20979347]]


In [32]:
### Regression Feature Selection

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

fs = SelectKBest(score_func=f_regression, k=4)

data_trans.shape

y.shape

# apply feature selection
X_selected = fs.fit_transform(data_trans, y)

parameters = fs.get_params(deep=True)

parameters

parameters['score_func']

print(X_selected.shape)

X_selected.shape[1]



df_rfe_selected = pd.DataFrame(X_selected)

df_rfe_selected.describe()



NameError: name 'y' is not defined